In [ ]:
from graphdatascience import GraphDataScience
from getpass import getpass

# Connect to Neo4j

In [ ]:
neo4j_password = getpass("Neo4j password")

In [ ]:
url = "neo4j+s://5f8297f1.databases.neo4j.io"
username = "neo4j"
gds = GraphDataScience(url, auth=(username, neo4j_password))

# Create Full text indexes
This is a one-time step to create the indexes you will need.

In [ ]:
def create_fulltext_index(index_name, label, property, analyzer):
    query = f"""
        CREATE FULLTEXT INDEX $indexName IF NOT EXISTS FOR (n:{label}) ON EACH [n.{property}]
        OPTIONS {{
            indexConfig: {{
                `fulltext.analyzer`: $analyzer,
                `fulltext.eventually_consistent`: true
            }}
        }}"""
    gds.run_cypher(query, {"indexName":index_name, "analyzer":analyzer})

create_fulltext_index("full_name_full_text", "PersonName", "fullName", "keyword")
create_fulltext_index("phone_number_full_text", "PhoneNumber", "phoneNumber", "keyword")
create_fulltext_index("ssn_full_text", "SocialSecurityNumber", "socialSecurityNumber", "keyword")
create_fulltext_index("email_full_text", "Email", "email", "email")
create_fulltext_index("dob_string_full_text", "DOB", "birthdateString", "keyword")

gds.run_cypher("show fulltext indexes")

# Create a function to query the full text indexes to find profiles that match at least two of the parameter values you provide.

In [ ]:
def search_profile(dob, email, full_name, phone, ssn):
    result_df = gds.run_cypher("""
        CALL {
            CALL db.index.fulltext.queryNodes("dob_string_full_text", $dob + "~", {limit:5}) YIELD node, score
            MATCH (u:UserDescription)-[:HAS_DOB]->(node)
            RETURN u, score, 
            {property: "dob", matchType: CASE WHEN node.birthdateString = $dob THEN "Exact" ELSE "Fuzzy" END} AS matched

            UNION ALL

            CALL db.index.fulltext.queryNodes("email_full_text", $email + "~", {limit:5}) YIELD node, score
            MATCH (u:UserDescription)-[:HAS_EMAIL]->(node)
            RETURN u, score, 
            {property: "email", matchType: CASE WHEN node.email = $email THEN "Exact" ELSE "Fuzzy" END} AS matched

            UNION ALL

            CALL db.index.fulltext.queryNodes("full_name_full_text", $fullName + "~", {limit:5}) YIELD node, score
            MATCH (u:UserDescription)-[:HAS_NAME]->(node)
            RETURN u, score, 
            {property: "name", matchType: CASE WHEN node.fullName = $fullName THEN "Exact" ELSE "Fuzzy" END} AS matched

                    
            UNION ALL

            CALL db.index.fulltext.queryNodes("phone_number_full_text", $phone + "~", {limit:5}) YIELD node, score
            MATCH (u:UserDescription)-[:HAS_PHONE]->(node)
            RETURN u, score, 
            {property: "phone", matchType: CASE WHEN node.phoneNumber = $phone THEN "Exact" ELSE "Fuzzy" END} AS matched

            UNION ALL

            CALL db.index.fulltext.queryNodes("ssn_full_text", $ssn + "~", {limit:5}) YIELD node, score
            MATCH (u:UserDescription)-[:HAS_SOCIAL_SECURITY_NUMBER]->(node)
            RETURN u, score, 
            {property: "ssn", matchType: CASE WHEN node.socialSecurityNumber = $ssn THEN "Exact" ELSE "Fuzzy" END} AS matched
        }

        WITH u, avg(score) AS avgScore, collect(matched) AS cluesMatched
        WHERE size(cluesMatched) > 1

        RETURN u.userId AS userId, 
        [clue IN cluesMatched WHERE clue['matchType'] = 'Exact' | clue['property']] AS exactMatchedProperties,
        [clue IN cluesMatched WHERE clue['matchType'] = 'Fuzzy' | clue['property']] AS fuzzyMatchedProperties,
        avgScore,
        COLLECT{ MATCH (u)-[:HAS_DOB]->(n) RETURN n.birthdateString } AS birthdates,
        COLLECT{ MATCH (u)-[:HAS_EMAIL]->(n) RETURN n.email } AS emails,
        COLLECT{ MATCH (u)-[:HAS_NAME]->(n) RETURN n.fullName } AS fullNames,
        COLLECT{ MATCH (u)-[:HAS_PHONE]->(n) RETURN n.phoneNumber } AS phoneNumbers,
        COLLECT{ MATCH (u)-[:HAS_SOCIAL_SECURITY_NUMBER]->(n) RETURN n.socialSecurityNumber } AS socialSecurityNumbers
            
        ORDER BY size(cluesMatched) DESC, avgScore DESC
                """,
                {"dob": dob, "email": email, "fullName": full_name, "phone": phone, "ssn": ssn})
    return result_df

# Test the function
## Exact match all properties

In [ ]:
search_profile("1930-08-28", "dolored.laborumb32@ex.com", "Belitd, Iutk Rincididunts", "268-023-9381", "996-32-0300")

## Exact match some properties and fuzzy match others

In [ ]:
search_profile("1933-08-28", "dolored.laborumb32@en.com", "Belitd, Iutk Rincididunts", "268-033-9381", "996-32-00300")

## Match three properties and mismatch others

In [ ]:
search_profile("1993-07-14", "xyz@abc.com", "Belitd, Iutk Rincididunts", "268-033-9381", "996-32-00300")